In [1]:
import pandas as pd
import numpy as np
sample_submission_df = pd.read_csv("./datasets/sample_submission_zingat.csv")
test_set = pd.read_csv("./datasets/zingat_usecase_testset_null.csv")
train_set = pd.read_csv("./datasets/zingat_usecase_trainset.csv")

## Change Variable

In [2]:
def change_variable(data):
    
    data = data.copy()
    return data.rename(columns = {
        'odasayısı': 'odasayisi',
        'brütm²': 'brutm2',
        'netm²': 'netm2',
        'binayaşı': 'binayasi',
        'bulunduğukat': 'bulundugukat',
        'binadakikatsayısı': 'binadakikatsayisi',
        'isıtmatipi': 'isitmatipi',
        'İnterkom': 'interkom',
        'depremyönetmeliğineuygun': 'depremyonetmeligineuygun',
        'asansör': 'asansor',
        'Çocukoyunalanı': 'cocukoyunalani',
        'giyinmeodası': 'giyinmeodasi',
        'güncel_fiyat': 'guncelfiyat',
    })
    return data

In [3]:
train_set = change_variable(train_set)
test_set = change_variable(test_set)

## Converting The Yes-None Values to 0-1

In [4]:
def convert_values(data):
    
    
    data.loc[data.interkom == "VAR", "interkom"] , data.loc[data.interkom == "YOK", "interkom"] = 1,0
    data.loc[data.depremyonetmeligineuygun == "VAR", "depremyonetmeligineuygun"] , data.loc[data.depremyonetmeligineuygun == "YOK", "depremyonetmeligineuygun"] = 1,0
    data.loc[data.asansor == "VAR", "asansor"] , data.loc[data.asansor == "YOK", "asansor"] = 1,0
    data.loc[data.cocukoyunalani == "VAR", "cocukoyunalani"] , data.loc[data.cocukoyunalani == "YOK", "cocukoyunalani"] = 1,0
    data.loc[data.giyinmeodasi == "VAR", "giyinmeodasi"] , data.loc[data.giyinmeodasi == "YOK", "giyinmeodasi"] = 1,0
    data.loc[data.ebeveynbanyosu == "VAR", "ebeveynbanyosu"] , data.loc[data.ebeveynbanyosu == "YOK", "ebeveynbanyosu"] = 1,0
    
    return data

In [5]:
train_set = convert_values(train_set)
test_set = convert_values(test_set)

## Convert Price

In [6]:
def convert_price(data):
    
    data["guncelfiyat"] = data.guncelfiyat.str[:-3]
    data["guncelfiyat"] = data["guncelfiyat"].astype(str).astype(int)
    return data

In [7]:
train_set = convert_price(train_set)

In [8]:
def tarih_fonksiyonu(data):
   
    data = data.copy()
    data[['yil','ay','gun']] = data["tarih"].str.split("-",expand=True)
   
    data["yil"] = data["yil"].astype(str).astype(int)
    data["ay"] = data["ay"].astype(str).astype(int)
    data["gun"] = data["gun"].astype(str).astype(int)
    return data
   

In [9]:
train_set = tarih_fonksiyonu(train_set)
test_set = tarih_fonksiyonu(test_set)

In [10]:
def belirtilmemis_fonksiyonu(data):
    
    data = data.copy()
    data['manzara'] = data['manzara'].replace("-","0")
    
    return data

In [11]:
train_set = belirtilmemis_fonksiyonu(train_set)
test_set = belirtilmemis_fonksiyonu(test_set)

In [12]:
def path_split_fonksiyonu(data):
    
    data = data.copy()
    data[['il','ilce','mahalle']] = data["path"].str.split("/",expand=True)
    
    return data

In [13]:
train_set = path_split_fonksiyonu(train_set)
test_set = path_split_fonksiyonu(test_set)

In [14]:
def odasayisi_fonksiyonu(data):
   
    data = data.copy()
    data['odasayisi'] = data['odasayisi'].replace("-","0+0")
    data[['odasayisi','salonsayisi']] = data['odasayisi'].str.split("+",expand=True)
    data["odasayisi"] = data["odasayisi"].astype(str).astype(int)
    data["salonsayisi"] = data["salonsayisi"].astype(str).astype(int)
    
    return data

In [15]:
train_set = odasayisi_fonksiyonu(train_set)
test_set = odasayisi_fonksiyonu(test_set)

In [16]:
label_encoder_cols = ("il","ilce","mahalle","emlaktipi","binayasi","bulundugukat","binadakikatsayisi","isitmatipi","otopark","manzara")

In [17]:
from sklearn.preprocessing import LabelEncoder
def label_encoder_fonksiyonu(data,cols):
    
    for col_name in label_encoder_cols:
        labelEncoder = LabelEncoder()
        labelEncoder.fit(data[col_name].values)
        data[col_name] = labelEncoder.transform(data[col_name].values)
        
    return data

In [18]:
train_set = label_encoder_fonksiyonu(train_set,label_encoder_cols)
test_set = label_encoder_fonksiyonu(test_set,label_encoder_cols)

In [19]:
def banyosayisi_fonksiyonu(data):
    
    data = data.copy()
    data['banyosayisi'] = data['banyosayisi'].replace("6 ve üzeri","6")
    data['banyosayisi'] = data['banyosayisi'].replace("-","-1")
    data['banyosayisi'] = data['banyosayisi'].astype(str).astype(int)
    
    return data

In [20]:
train_set = banyosayisi_fonksiyonu(train_set)
test_set = banyosayisi_fonksiyonu(test_set)

In [21]:
def object_to_int(data):
    
    data = data.copy()
    data[['interkom','depremyonetmeligineuygun','asansor','cocukoyunalani','giyinmeodasi','ebeveynbanyosu']] = data[['interkom',
                                                                                                                     'depremyonetmeligineuygun',
                                                                                                                     'asansor','cocukoyunalani',
                                                                                                                     'giyinmeodasi','ebeveynbanyosu']].astype(str).astype(int)
    
    return data

In [22]:
train_set = object_to_int(train_set)
test_set = object_to_int(test_set)

In [23]:
train_set = train_set.drop(['tarih','path'], axis = 1)
test_set = test_set.drop(['tarih','path'], axis = 1)

In [24]:
train_set.to_csv('./datasets/final_train_set.csv')
test_set.to_csv('./datasets/final_test_set.csv')